### Evaluate

In [1]:
import os
import torch

In [2]:
model_path = "./model/wav2vec"

In [3]:
# GPU
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = '6,7'

In [4]:
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(model_path)
model = Wav2Vec2ForCTC.from_pretrained(model_path).to(device)

/home/haram/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/haram/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [5]:
import soundfile as sf
from datasets import load_from_disk

In [6]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def map_to_result(batch):
    # print("batch path",batch["path"])
    # audio 처리
    batch["speech"], _ = sf.read(batch["path"])

    # 전사 데이터 처리
    result = re.sub(chars_to_ignore_regex, "", batch["transcription"])
    result = " ".join(result.split())
    result = result.lower() 
    batch["result"] = result

    return batch

In [7]:
def prepare_example(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to(device)
    
    with torch.no_grad():
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
    
    predicted_ids[predicted_ids == -100] = processor.tokenizer.pad_token_id
    predicted_transcripts = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

    batch["predicted_text"] = predicted_transcripts
    
    return batch

In [8]:
import random
import pandas as pd
from IPython.display import display, HTML

# 무작위로 샘플 보여주는 함수 정의
def show_random_elements(dataset, num_examples=7):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [9]:
from jiwer import wer

def eval_model(dataset_path):
    dataset = load_from_disk(dataset_path)

    dataset = dataset.map(map_to_result,remove_columns=["transcription","audio","path"])
    dataset = dataset.map(prepare_example, batched=True, batch_size=16)

    show_random_elements(dataset,5)

    print(wer(dataset["result"],dataset["predicted_text"]))

In [10]:
# eval_model("./data/datasets/gaussian-10")

In [11]:
# eval_model("./data/datasets/gaussian-20")

In [12]:
# eval_model("./data/datasets/gaussian-30")

In [13]:
# eval_model("./data/datasets/gaussian-40")

In [14]:
# eval_model("./data/datasets/gaussian-50")

In [15]:
eval_model("./data/datasets/gaussian-60")

Parameter 'function'=<function map_to_result at 0x7f88129f5cf0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
11350ex [01:53, 100.08ex/s]
  0%|          | 0/710 [00:00<?, ?ba/s]2024-04-26 08:14:59.788509: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 08:14:59.834527: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register

,speech,result,predicted_text
0,"[-0.02703857421875, 0.0126953125, 0.021453857421875, -0.0020751953125, -0.02020263671875, 0.00201416015625, -0.0257568359375, 0.00946044921875, 0.014984130859375, -0.00274658203125, -0.013519287109375, 0.02490234375, 0.021148681640625, 0.005706787109375, 0.024017333984375, 0.0032958984375, -0.0391845703125, 0.00341796875, -0.00341796875, -0.00799560546875, 0.02911376953125, 0.00933837890625, 0.01092529296875, -0.022796630859375, 0.0286865234375, 0.014984130859375, -0.002197265625, 0.0264892578125, -0.015380859375, -0.002471923828125, 0.02435302734375, 0.005767822265625, 0.032562255859375, -0.0218505859375, -0.03955078125, -0.025421142578125, -0.009765625, -0.05499267578125, -0.00860595703125, -0.02337646484375, -0.016326904296875, 0.00567626953125, -0.00994873046875, 0.040252685546875, 0.018341064453125, -0.00372314453125, -0.017791748046875, -0.01080322265625, 0.019134521484375, -0.001068115234375, -0.031951904296875, 0.013397216796875, 0.020233154296875, -0.005279541015625, 0.025421142578125, 0.03167724609375, -0.001190185546875, -0.015655517578125, 0.00921630859375, 0.01226806640625, -0.0128173828125, -0.002716064453125, -0.016815185546875, 0.030670166015625, 0.019989013671875, 0.007781982421875, 0.003936767578125, 0.0301513671875, -0.000335693359375, 0.0003662109375, 0.001068115234375, -0.010589599609375, 0.0224609375, -0.010467529296875, -0.023895263671875, -0.017120361328125, 0.014007568359375, 0.03997802734375, -0.024322509765625, 0.0323486328125, -0.01373291015625, 0.008941650390625, 0.024658203125, -0.030487060546875, 0.012725830078125, -0.002349853515625, 0.003173828125, 0.004150390625, 0.007598876953125, 0.0240478515625, -0.01239013671875, 0.039031982421875, 0.019439697265625, -0.014862060546875, 0.04833984375, -0.008270263671875, -0.002532958984375, 0.00909423828125, 0.031097412109375, 0.006805419921875, ...]",when he had thus amassed them together he took a warehouse which happened to be opposite to cassim's which ali baba's son had occupied since the death of his uncle,when he had thus amasd them to gether whe to aweraswhich hapened to be opose se which holivolu's son had ocupie s o defofis o
1,"[-0.062225341796875, 0.043304443359375, -0.03515625, -0.00811767578125, -0.018218994140625, -0.02362060546875, 0.06658935546875, -0.039215087890625, -0.0338134765625, 0.042816162109375, -0.029052734375, 0.062713623046875, -0.044769287109375, -0.0008544921875, 0.01971435546875, 0.02130126953125, -0.01318359375, -0.01959228515625, -0.0133056640625, -0.06005859375, 0.0047607421875, 0.08892822265625, 0.00115966796875, 0.014373779296875, 0.008941650390625, 0.020538330078125, -0.0150146484375, 0.049346923828125, -0.0406494140625, -0.049285888671875, 0.035797119140625, 0.027374267578125, -0.04296875, 0.021759033203125, 0.00421142578125, 0.003143310546875, -0.0711669921875, -0.048004150390625, 0.0284423828125, 0.042816162109375, -0.002899169921875, 0.00457763671875, -0.024658203125, -0.011749267578125, 0.091888427734375, -0.054840087890625, -0.075469970703125, -0.061676025390625, 0.026153564453125, 0.048095703125, -0.02679443359375, -0.00482177734375, -0.0006103515625, 0.018890380859375, 0.058380126953125, 0.0103759765625, 0.010955810546875, -0.026763916015625, 0.003204345703125, 0.03631591796875, 0.00811767578125, -0.061798095703125, 0.025390625, -0.01287841796875, 0.042449951171875, -0.042144775390625, -0.007781982421875, 0.020721435546875, -0.05511474609375, -0.00921630859375, 0.072357177734375, 0.001129150390625, -0.12542724609375, -0.003387451171875, -0.00531005859375, -0.00030517578125, 0.02978515625, -0.032440185546875, 0.053314208984375, -0.000518798828125, 0.013519287109375, -0.057525634765625, -0.00531005859375, 0.034271240234375, -0.09771728515625, 0.048828125, 0.0552978515625, -0.019439697265625, 0.0072021484375, -0.01422119140625, 0.02081298828125, 0.01861572265625, 0.008056640625, 0.042755126953125, -0.019073486328125, 0.04083251953125, 0.027069091796875, -0.061737060546875, 0.007720

0.8057080075629822


In [16]:
eval_model("./data/datasets/gaussian-70")

11350ex [01:52, 101.28ex/s]
100%|██████████| 710/710 [18:01<00:00,  1.52s/ba]


,speech,result,predicted_text
0,"[0.067779541015625, -0.023284912109375, 0.0079345703125, 0.05755615234375, -0.006927490234375, 0.08038330078125, 0.101776123046875, 0.01776123046875, -0.00262451171875, -0.065673828125, -0.102081298828125, -0.03082275390625, 0.05059814453125, 0.014801025390625, 0.019561767578125, 0.0712890625, -0.03143310546875, -0.023712158203125, 0.023040771484375, 0.007415771484375, -0.025726318359375, 0.008514404296875, -0.052337646484375, -0.11602783203125, -0.06768798828125, 0.012786865234375, -0.053131103515625, 0.034088134765625, 0.0181884765625, -0.05023193359375, -0.03436279296875, -0.0108642578125, 0.012786865234375, 0.0804443359375, -0.0399169921875, 0.006072998046875, -0.021148681640625, 0.006317138671875, -0.021636962890625, -0.016876220703125, 0.10760498046875, 0.03009033203125, 0.048004150390625, -0.006591796875, 0.068695068359375, 0.036376953125, 0.067657470703125, -0.020477294921875, -0.06353759765625, -0.11041259765625, 0.020843505859375, -0.02685546875, -0.017486572265625, 0.056304931640625, 0.05517578125, 0.01904296875, 0.008392333984375, 0.002593994140625, 0.04388427734375, -0.041717529296875, 0.01416015625, -0.02099609375, 0.09228515625, 0.015655517578125, 0.003082275390625, -0.0203857421875, 0.002716064453125, -0.055389404296875, -0.032196044921875, -0.034149169921875, 0.0093994140625, 0.027984619140625, 0.027587890625, 0.060638427734375, 0.04541015625, 0.00836181640625, -0.047210693359375, -0.060516357421875, -0.019439697265625, -0.01910400390625, 0.0184326171875, -0.0079345703125, 0.02813720703125, -0.043426513671875, 0.0181884765625, 0.015350341796875, 0.032684326171875, -0.056854248046875, 0.040863037109375, -0.013275146484375, -0.078338623046875, -0.036407470703125, 0.0078125, 0.031829833984375, -0.06304931640625, -0.01788330078125, -0.03271484375, -0.00506591796875, -0.015472412109375, 0.00225830078125, ...]",at the magic touch of the beautiful the secret chords of our being are awakened we vibrate and thrill in response to its call mind speaks to mind we listen to the unspoken we gaze upon the unseen,an i ie eior a ie eie r i n e ia i
1,"[0.128509521484375, 0.1632080078125, 0.129180908203125, 0.142120361328125, 0.138153076171875, 0.110015869140625, 0.140472412109375, 0.154754638671875, 0.150238037109375, 0.160003662109375, 0.194000244140625, 0.11065673828125, 0.153076171875, 0.158843994140625, 0.135284423828125, 0.208648681640625, 0.1829833984375, 0.11724853515625, 0.188568115234375, 0.19256591796875, 0.150665283203125, 0.123687744140625, 0.151824951171875, 0.10650634765625, 0.13189697265625, 0.183868408203125, 0.1387939453125, 0.164093017578125, 0.164703369140625, 0.10546875, 0.188720703125, 0.14495849609375, 0.17315673828125, 0.1563720703125, 0.128509521484375, 0.17633056640625, 0.17181396484375, 0.175048828125, 0.144287109375, 0.121124267578125, 0.153717041015625, 0.15618896484375, 0.164306640625, 0.138275146484375, 0.102142333984375, 0.103546142578125, 0.135162353515625, 0.094879150390625, 0.10333251953125, 0.119873046875, 0.177337646484375, 0.116973876953125, 0.146881103515625, 0.1199951171875, 0.139739990234375, 0.100250244140625, 0.141265869140625, 0.170623779296875, 0.152008056640625, 0.1959228515625, 0.177734375, 0.1424560546875, 0.13555908203125, 0.183807373046875, 0.11224365234375, 0.1036376953125, 0.128143310546875, 0.171112060546875, 0.1441650390625, 0.11297607421875, 0.129180908203125, 0.095794677734375, 0.129241943359375, 0.137908935546875, 0.11614990234375, 0.143035888671875, 0.1824951171875, 0.134307861328125, 0.128082275390625, 0.13800048828125, 0.15472412109375, 0.14111328125, 0.16650390625, 0.12518310546875, 0.104217529296875, 0.117950439453125, 0.157958984375, 0.10638427734375, 0.144317626953125, 0.1036376953125, 0.111602783203125, 0.1605224609375, 0.10919189453125, 0.14764404296875, 0.127655029296875, 0.1614990234375, 0.11676025390625, 0.155853271484375, 0.127716064453125, 0.10906982421875, ...]",he was a hop fiend all right still there's nothing to indicate 

0.8804844396749961


In [ ]:
eval_model("./data/datasets/gaussian-80")

 99%|█████████▉| 703/710 [18:01<00:10,  1.50s/ba]

In [18]:
eval_model("./data/datasets/gaussian-90")

11350ex [01:56, 97.27ex/s] 
100%|██████████| 710/710 [18:08<00:00,  1.53s/ba]


,speech,result,predicted_text
0,"[0.011871337890625, 0.09393310546875, 0.04644775390625, 0.031646728515625, -0.020660400390625, -0.160247802734375, -0.07666015625, 0.084808349609375, -0.148773193359375, 0.07574462890625, -0.128662109375, -0.028533935546875, -0.002685546875, 0.013763427734375, 0.0076904296875, 0.055267333984375, 0.141448974609375, 0.030670166015625, -0.022125244140625, 0.161041259765625, -0.006256103515625, 0.019073486328125, -0.0447998046875, -0.033172607421875, -0.03765869140625, -0.0072021484375, 0.085418701171875, -0.044036865234375, -0.029754638671875, -0.015533447265625, -0.030487060546875, 0.1044921875, -0.02783203125, 0.06036376953125, 0.044342041015625, -0.16680908203125, 0.11248779296875, -0.12994384765625, 0.053131103515625, -0.135101318359375, -0.022613525390625, -0.07440185546875, -0.011505126953125, -0.044097900390625, -0.01324462890625, 0.132415771484375, 0.026458740234375, 0.0118408203125, -0.04132080078125, -0.093231201171875, -0.05511474609375, -0.087738037109375, -0.05511474609375, -0.02105712890625, 0.02703857421875, 0.003021240234375, -0.029876708984375, 0.115264892578125, -0.0537109375, -0.037933349609375, 0.002838134765625, -0.094451904296875, -0.032989501953125, -0.019775390625, -0.05487060546875, -0.046600341796875, 0.058929443359375, -0.061920166015625, -0.025390625, 0.062469482421875, -0.11297607421875, -0.0511474609375, -0.065460205078125, -0.040771484375, 0.10601806640625, 0.016571044921875, -0.045867919921875, -0.003692626953125, -0.064666748046875, -0.04241943359375, -0.05377197265625, -0.01202392578125, 0.03302001953125, -0.032958984375, 0.044281005859375, 0.0899658203125, 0.028076171875, 0.00628662109375, -0.002899169921875, -0.053802490234375, 0.0865478515625, -0.052703857421875, -0.004119873046875, -0.006256103515625, 0.146759033203125, 0.006927490234375, -0.01800537109375, 0.063385009765625, -0.06927490234375, -0.00775146484375, ...]",and seemed but barely under the control of their depleted crews their fire had ceased entirely and all their energies seemed focused upon escape our warriors then rushed up to the roofs of the buildings which we occupied,inse t fery a ae eire ehir aera aehowoer enuaer
1,"[-0.02239990234375, 0.070709228515625, 0.07989501953125, -0.06378173828125, -0.021697998046875, 0.010162353515625, -0.029388427734375, -0.0372314453125, 0.001434326171875, -0.026336669921875, -0.0531005859375, -0.031219482421875, 0.082427978515625, 0.012176513671875, -0.0025634765625, -0.162139892578125, 0.001983642578125, -0.1126708984375, -0.04107666015625, -0.08355712890625, 0.034637451171875, -0.07012939453125, 0.079193115234375, -0.08905029296875, -0.0174560546875, -0.01202392578125, 0.038482666015625, 0.029541015625, 0.13702392578125, 0.025177001953125, 0.05615234375, 0.0281982421875, 0.116912841796875, -0.014556884765625, -0.118621826171875, -0.066986083984375, 0.025146484375, 0.01202392578125, -0.01568603515625, 0.00079345703125, -0.02239990234375, -0.0194091796875, -0.0316162109375, -0.008819580078125, 0.06878662109375, 0.037506103515625, 0.00872802734375, 0.024383544921875, -0.049835205078125, 0.07867431640625, 0.05853271484375, 0.023529052734375, -0.007720947265625, -0.097808837890625, 0.076934814453125, -0.146240234375, -0.1026611328125, -0.042388916015625, -0.076385498046875, 0.0616455078125, 0.014068603515625, -0.09033203125, 0.078338623046875, 0.131805419921875, -0.076324462890625, -0.029541015625, 0.03826904296875, -0.00164794921875, 0.0069580078125, 0.024017333984375, 0.02667236328125, 0.1025390625, 0.007568359375, -0.04901123046875, -0.04364013671875, 0.005157470703125, 0.03521728515625, 0.0753173828125, 0.029144287109375, -0.0255126953125, -0.004791259765625, 0.00616455078125, 0.100921630859375, -0.07958984375, -0.04046630859375, -0.000152587890625, -0.047698974609375, 0.0120849609375, 0.08154296875, -0.02435302734375, -0.010406494140625, -0.050445556640625, 0.01373291015625, -0.05072021484375, 0.186370849609375, 0.020294189453125, -0.108245849609375, -0.00817

0.9591343451377178


In [19]:
eval_model("./data/datasets/gaussian-100")

11350ex [02:04, 91.41ex/s] 
100%|██████████| 710/710 [18:01<00:00,  1.52s/ba]


,speech,result,predicted_text
0,"[0.073394775390625, -0.0408935546875, -0.0465087890625, -0.022247314453125, 0.046234130859375, -0.027862548828125, -0.095428466796875, -0.052886962890625, 0.03948974609375, -0.009552001953125, -0.020843505859375, -0.043487548828125, 0.034576416015625, 0.06817626953125, 0.008270263671875, -0.22894287109375, 0.0064697265625, 0.019561767578125, -0.0726318359375, -0.104034423828125, 0.10272216796875, 0.00634765625, -0.21484375, -0.005615234375, -0.01605224609375, -0.014129638671875, 0.02423095703125, 0.081146240234375, 0.10797119140625, 0.0152587890625, 0.11932373046875, 0.087127685546875, 0.044464111328125, -0.04388427734375, 0.151153564453125, 0.170074462890625, -0.09478759765625, 0.031402587890625, 0.1231689453125, 0.231536865234375, -0.142059326171875, -0.066131591796875, -0.046875, 0.11602783203125, -0.047943115234375, 0.04376220703125, 0.15753173828125, -0.049652099609375, -0.243011474609375, 0.149078369140625, 0.074981689453125, -0.094635009765625, -0.006500244140625, 0.160125732421875, -0.00390625, 0.004730224609375, 0.079833984375, -0.044769287109375, -0.056549072265625, 0.068267822265625, -0.148712158203125, 0.218994140625, -0.1724853515625, -0.154510498046875, -0.095977783203125, 0.01153564453125, -0.18475341796875, 0.02447509765625, 0.044464111328125, -0.103515625, -0.07476806640625, 0.084564208984375, 0.00244140625, -0.08099365234375, -0.05267333984375, -0.066680908203125, 0.10919189453125, 0.019500732421875, -0.067626953125, -0.03594970703125, -0.06512451171875, 0.04791259765625, 0.084625244140625, 0.09100341796875, 0.065887451171875, 0.03497314453125, -0.0008544921875, 0.0133056640625, 0.173583984375, -0.0048828125, -0.081451416015625, -0.019500732421875, 0.036529541015625, -0.052520751953125, -0.0908203125, -0.0740966796875, 0.0228271484375, -0.14361572265625, -0.004241943359375, 0.042816162109375, ...]",and fell to prodding and slashing them with lance and sword,yi
1,"[0.105712890625, 0.0384521484375, 0.058135986328125, 0.014495849609375, 0.0615234375, 0.06036376953125, 0.039764404296875, 0.17401123046875, 0.0462646484375, 0.079132080078125, 0.01629638671875, 0.0950927734375, -0.073150634765625, -0.029510498046875, 0.09027099609375, 0.02197265625, 0.10748291015625, 0.007598876953125, 0.10186767578125, 0.064117431640625, -0.00518798828125, 0.13470458984375, 0.011505126953125, 0.0755615234375, 0.068359375, 0.09326171875, 0.018096923828125, 0.149658203125, 0.0401611328125, 0.101531982421875, -0.0030517578125, 0.072845458984375, 0.068603515625, 0.110626220703125, 0.095733642578125, 0.069061279296875, 0.074554443359375, 0.011993408203125, 0.098602294921875, 0.109344482421875, 0.07940673828125, 0.104522705078125, 0.1727294921875, 0.134124755859375, 0.10479736328125, 0.066192626953125, 0.12591552734375, 0.040435791015625, 0.160003662109375, 0.090667724609375, 0.01568603515625, 0.08917236328125, 0.058624267578125, 0.1031494140625, 0.101318359375, 0.140869140625, 0.043487548828125, 0.02020263671875, 0.05462646484375, 0.0640869140625, 0.041900634765625, 0.107421875, 0.123138427734375, 0.1524658203125, 0.035491943359375, -0.004638671875, 0.11322021484375, 0.062255859375, 0.04571533203125, 0.015838623046875, 0.033294677734375, 0.0455322265625, 0.091278076171875, 0.098297119140625, 0.079833984375, 0.14404296875, 0.128021240234375, 0.119781494140625, 0.04150390625, 0.099609375, 0.125823974609375, 0.0211181640625, 0.008575439453125, 0.060150146484375, 0.020782470703125, 0.04766845703125, 0.04754638671875, 0.042572021484375, 0.09814453125, 0.0601806640625, 0.05499267578125, 0.105682373046875, 0.0723876953125, 0.099395751953125, 0.002960205078125, 0.094146728515625, 0.1016845703125, 0.08349609375, 0.09344482421875, 0.056427001953125, ...]",but if she is determined to have her feelings hurt it can't be helped if i were to go to the shore some night with you do you think i could see your rock people too paul shook his head gravely no i don't think you could see my rock people,but it's eo en ever uriy

0.9771143134549543
